#### Simulation Of Scenarios

In [62]:
## Import Packages
import itertools
import random


### Torch Packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split # For Subset Selection

### Triple-Gamma-Reg
import tgr

In [73]:
## Set Scenario Values
dataset_size = [10] # In Integers; as in N = 100
sparsity_values = [0.5] # In Percentage; as in 10% of coefficients are different from 0
num_of_coefs = [5] # In Integers; as in total number of coefficients is 10

scenarios = list(itertools.product(dataset_size, sparsity_values, num_of_coefs))

In [70]:
## Create class to better handle scenario datasets
class ScenarioDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [85]:
## Simulation
torch.manual_seed(12345)
random.seed(12345)
torch.set_printoptions(sci_mode=False, precision=10)

for scenario in scenarios:
    print(f'Data Set Size: {scenario[0]}; Sparsity: {scenario[1]*100}%; Number of Coefficients: {scenario[2]}')
    
    ## Retrieve Simulation Parameters
    sample = scenario[0]
    sparsity = scenario[1]
    coefs = scenario[2]
    
    ## Get True Coefficients for Data generating Process
    coefs_list = int(sparsity*coefs) * [0] # Create List of Zero Coefficients
    coefs_list.extend(round(random.uniform(-5, 5),5) for _ in range(coefs - int(sparsity*coefs))) # Fill up remaining Coeficients with random parameters
    random.shuffle(coefs_list) # Shuffle the List
    #print(coefs_list)
    
    ## Create Data based on True Coefficients
    X = torch.randn(sample, coefs)
    Y = X @ torch.tensor(coefs_list) + torch.randn(sample, 1) * 0.1
    
    ## Split dataset randomly into Train and Test Data
    train_percentage = 0.8 # Amount of dataset which is used for training data
    dataset = ScenarioDataset(X, Y)
    train_dataset, test_dataset = random_split(dataset, [int(train_percentage*sample),sample-int(train_percentage*sample)])
    
    print(test_dataset.dataset.Y)
    
    
    ### OLS Regression
    testerror_OLS = list()
    
    ### TGR Regression
    testerror_TGR = list()
    
    ### LASSO Regression
    testerror_LASSO = list()
    
    ### Ridge Regression
    testerror_RIDGE = list()
    
    ### Arctan Regression
    testerror_ARCTAN = list()
    
    ### Gaussian Regression
    testerror_GAUSSIAN = list()
    
    ## Output in Table
    testerror_scenario = [testerror_OLS, testerror_TGR, testerror_LASSO, testerror_RIDGE, testerror_ARCTAN, testerror_GAUSSIAN]
    
    


Data Set Size: 10; Sparsity: 50.0%; Number of Coefficients: 5
tensor([[ 8.3477144241,  6.1906638145, -2.6268305779, -2.0533258915,
         -2.3993585110,  6.9432682991, -3.3761820793, -0.7660281658,
          5.1017742157,  2.4091131687],
        [ 8.3815832138,  6.2245326042, -2.5929615498, -2.0194568634,
         -2.3654894829,  6.9771370888, -3.3423130512, -0.7321591973,
          5.1356430054,  2.4429821968],
        [ 8.4041728973,  6.2471213341, -2.5703728199, -1.9968680143,
         -2.3429007530,  6.9997258186, -3.3197243214, -0.7095702887,
          5.1582317352,  2.4655709267],
        [ 8.2987489700,  6.1416978836, -2.6757962704, -2.1022915840,
         -2.4483242035,  6.8943023682, -3.4251477718, -0.8149938583,
          5.0528082848,  2.3601474762],
        [ 8.3452510834,  6.1882004738, -2.6292936802, -2.0557889938,
         -2.4018216133,  6.9408049583, -3.3786451817, -0.7684913278,
          5.0993108749,  2.4066500664],
        [ 8.4221277237,  6.2650761604, -2.552417

C:\Users\Lucas Paul\AppData\Local\Temp\ipykernel_27884\1756608188.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)
C:\Users\Lucas Paul\AppData\Local\Temp\ipykernel_27884\1756608188.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.Y = torch.tensor(Y, dtype=torch.float32)
